In [1]:
from dataclasses import dataclass
from math import gcd, ceil
import re
import numpy as np
from collections import Counter, defaultdict, namedtuple, deque, OrderedDict
from functools import cache
from itertools import product, chain, combinations, combinations_with_replacement, permutations, accumulate, zip_longest, batched, pairwise
import numpy as np
from matplotlib import pyplot as plt
import networkx as nx

from aocutils.common import copy_func, patch_to, patch, to_int, ints, flatten, zippify, multidict, rev, data, quantify, atom, atoms, list_atoms, list_multiply, mapt
from aocutils.grid import gridneigh, arr_to_dict, grid_to_dict, neighbors, complexneighbors, arr_neighbors, iterate, Dim, dimensions, positive, manhattan, conv1d, conv2d, rotate, E, W, N, S, winds, TURNLEFT, TURNRIGHT
from aocutils.maze import graphparse, bfs, dijkstra, distances, get_path, dfs
from aocutils.math import gcd, factors, lcm, crt, mul_inv, Segment, lis, angle, all_combinations, all_permutations, mst, shoelace
from aocutils.special import md5, binarysearch, deduce_matches, find_specific_pattern, find_repeat, find_cycle, UnionFind, Octree, LazySegmentTree, Trie
from aocutils.visuals import visualize_graph, labelize, animate_grid, plot, animate
from aocutils.cfg import CFG
from aocutils.earley import State, Earley
from aocutils.shunting import ShuntingYard

inf = float('inf')

In [8]:
def move(loc, direc):
    return (loc[0] + direc[0], loc[1] + direc[1])

In [40]:
f = open("in.txt").read().split("\n")
lines = [list(line) for line in f]
lines[0]
points = set()
conn = defaultdict(set)
cars = []

bombs = set()

go = {'>': E, '<':W, '^':N, 'v':S}
for r,c, val in iterate(lines):
    if val != '':
        points.add((r,c))
    if val in '><^v':
        cars.append((r,c,go[val], 0))
        bombs.add((r,c))
    if val in '<>':
        lines[r][c] = '-'
    if val in '^v':
        lines[r][c] = '|'
        

def changedir(newloc, val, time):
    line =  lines[newloc[0]][newloc[1]]
    if line == '+':
        if time == 0:
            newval = TURNLEFT(val)
        elif time == 1:
            newval = val
        elif time == 2:
            newval = TURNRIGHT(val)
        else:
            print('error')
        time = (time + 1) % 3
    elif line == '\\':
        if val == E: newval = S
        elif val == N: newval = W
        elif val == S: newval = E
        elif val == W: newval = N
    elif line == '/':
        if val == E: newval = N
        elif val == N: newval = E
        elif val == S: newval = W
        elif val == W: newval = S
    else:
        newval = val
    return newval, time
    
for _ in range(15000):
    
    forbidden = set()
    newcars = []
    print(bombs, 'new roudn')
    for r,c,val, time in sorted(cars):
        if (r,c) in forbidden:
            # print('all crashed')
            continue
        else:
            bombs.remove((r,c))
            newloc = move((r,c), val)
            if newloc in bombs:
                # print(newloc,'crash')
                bombs.remove(newloc)
                forbidden.add(newloc)
                for new in newcars[:]:
                    if new[0] == newloc[0] and new[1] == newloc[1]:
                        newcars.remove(new)
                        
            else:
                # print('add')
                newcars.append((newloc[0],newloc[1], *changedir(newloc, val, time)))
                assert (newloc[0], newloc[1]) in points
                cars.remove((r,c,val, time))
                bombs.add(newloc)
    cars = newcars
    # print(len(bombs), bombs, cars)
    if len(bombs) == 1:
        break
bombs
        
        
        

{(120, 103), (76, 117), (41, 49), (69, 110), (80, 59), (117, 39), (94, 77), (126, 114), (73, 93), (107, 51), (23, 147), (99, 12), (115, 12), (113, 114), (146, 33), (83, 114), (82, 33)} new roudn
{(70, 110), (83, 113), (146, 32), (118, 39), (100, 12), (73, 92), (82, 34), (75, 117), (94, 78), (24, 147), (80, 60), (42, 49), (126, 115), (107, 50), (113, 115), (120, 102), (114, 12)} new roudn
{(107, 49), (94, 79), (79, 60), (120, 101), (25, 147), (146, 31), (42, 50), (126, 116), (113, 116), (71, 110), (84, 113), (113, 12), (81, 34), (74, 117), (119, 39), (101, 12), (74, 92)} new roudn
{(126, 117), (85, 113), (42, 51), (74, 116), (80, 34), (107, 48), (120, 39), (120, 100), (101, 13), (75, 92), (146, 30), (78, 60), (26, 147), (94, 80), (72, 110), (112, 116), (112, 12)} new roudn
{(76, 92), (121, 39), (146, 29), (101, 14), (111, 116), (26, 148), (74, 115), (111, 12), (73, 110), (94, 81), (86, 113), (125, 117), (79, 34), (107, 47), (42, 52), (120, 99), (77, 60)} new roudn
{(124, 117), (26, 149)

{(81, 65)}

In [7]:
E = 1; W = -1; N = 1j; S = -1j
TURNLEFT = lambda x: x * 1j
TURNRIGHT = lambda x: x * -1j

# parse grid with complex numbers
f = open("in.txt").read().split("\n")
lines = [list(line.split()) for line in f]
grid = arr_to_dict(lines, tocomplex=True)
complexneighbors(3+2j)

{(2+2j), (3+1j), (3+3j), (4+2j)}